# Imports

In [179]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import json

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# functions

def request_soup(url_link):    
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0'}    
    page = requests.get( url, headers = headers)
    soup_obj = BeautifulSoup(page.text, 'html.parser')
    return( soup_obj )

def composition_to_df(list_of_comp):
    """ Creates a dataframe from a list of compositions"""
    keys = []
    values = []

    for idx, element in enumerate(list_of_comp):
        # if idx is even, element is a key (column in dataframe)
        if idx % 2 == 0:
            keys.append(element)
        else:
            values.append(element.strip('%,'))

    # final dataframe
    res = dict(zip(keys,values))
    res = pd.DataFrame(res, index = [0])
    return (res)

# Data Requesting

##  Home Page Scraping

In [4]:
# all products url
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# headers for request
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X x.y; rv:42.0) Gecko/20100101 Firefox/42.0'}

# requesting
page = requests.get(url=url, headers=headers)

# instatiating bs4 object
soup = BeautifulSoup(page.text, 'html.parser')

In [5]:
# finding load more products element
p = soup.find('div', class_='load-more-products')

# all products
all_products = int(p.find('h2').get('data-total'))

# products per page
products_per_page = int(p.find('h2').get('data-items-shown'))

# rounding up numer of pages needed for web scraping
total_pages = np.ceil(all_products/products_per_page)


##  All products in Home Page Scraping

In [6]:
# creating a page with all products
url_all_prods = url + '?&offset=0&page-size={}'.format(int(total_pages*products_per_page))

all_prods = requests.get(url = url_all_prods, headers=headers)

In [7]:
soup = BeautifulSoup(all_prods.text, 'html.parser')#.get('li', class_='product-item')

# soup.find('li', class_ = 'product-item').find('a').get('href') #.get('item-link')  #.get('item-link') #, class_ = 'item-link')
# all find all products listed in homepage
products = soup.find_all('li', class_='product-item')

# get link to all projects
home_links = ['https://www2.hm.com' + link.find('a').get('href') for link in products ]

##  All products in Each Product Page

In [8]:
# resulting list of all products to scrap
links = []

for link in home_links:
    # scrap each product in home page list
    single_product = requests.get(link, headers = headers)
    soup = BeautifulSoup(single_product.text, 'html.parser')

    # gets the links to all products listed in a page
    products_ul = soup.find('ul', class_='inputlist clearfix')
    products = products_ul.find_all('a')

    links_ul = []
    links_ul = [ 'https://www2.hm.com' + item.get('href') for item in products]
    links.extend(links_ul)

In [9]:
# getting all unique products listed

# converting to a set and then back to list
links = list(set(links))
links.sort()

In [10]:
# defining base dataframe
df_prods = pd.DataFrame()

for link in links:
    
    # scrap each product in home page list
    single_product = requests.get(link, headers = headers)
    soup = BeautifulSoup(single_product.text, 'html.parser')
    
    # scrap all products listed in a page
    products_ul = soup.find('ul', class_='inputlist clearfix')
    products = products_ul.find_all('a')

    # product headline
    headline = soup.find('h1', class_='primary product-item-headline').text


    for product in products:
        
        #product it
        sku = product.get('data-articlecode')
       
        # color
        color = product.get('data-color')
        
        # product id
        product_id = sku[:-3]
        
        # style id
        color_id = sku[-3:]

        # link
        link = 'https://www2.hm.com/en_us/productpage.{}.html'.format(sku)

        df_temp = pd.DataFrame( {'sku': sku, 'product_id' :product_id, 'color_id' : color_id, 'color': color, 'headline' : headline, 'link': link}, index = [0] )
        
        df_prods = pd.concat([df_prods, df_temp], axis = 0)


df_prods.drop_duplicates('sku',inplace = True)

df_prods.reset_index(inplace = True, drop = True)

In [11]:
df_prods.head()

,sku,product_id,color_id,color,headline,link
0,0427159001,0427159,001,Black denim,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
1,0427159002,0427159,002,Blue washed out,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
2,0427159003,0427159,003,Denim blue,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
3,0427159004,0427159,004,Light denim blue,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...
4,0427159005,0427159,005,Dark denim blue,\n\t\t\t\t\t\t\t Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...


## Individual Scraping

### Scraping Data

In [12]:
# starting drive

options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [13]:
links[0]

'https://www2.hm.com/en_us/productpage.0427159001.html'

### Product Description

In [14]:
driver.get("https://www2.hm.com/en_us/productpage.0811993021.html")

# class_ = "BodyText-module--general__32l6J" # if below doesn't work
class_ = "ProductDescription-module--descriptionText__1zy9P"

try: 
    content = WebDriverWait(driver, 10).until(EC.presence_of_element_located( (By.CLASS_NAME, class_) ))
    desc = content.text
except:
    desc = 'NA'

content = WebDriverWait(driver, 10).until(EC.presence_of_element_located( (By.CLASS_NAME, class_) ))
desc = content.text
desc

'H&M Essentials. No. 2: The Jeans. 5-pocket jeans in stretch cotton denim. Regular waist, zip fly with button, and straight legs with good room for movement over thighs and knees.'

### Text

In [15]:
# driver.get(links[0])
driver.get("https://www2.hm.com/en_us/productpage.1024256004.html")

In [16]:
elements = driver.find_elements(by=By.CLASS_NAME, value="ProductAttributesList-module--descriptionListItem__3vUL2")
for e in elements:
    print(e.text)

FitSlim fit
CompositionShell: Cotton 99%, Spandex 1%Pocket lining: Polyester 65%, Cotton 35%
More sustainable materialsShell: Recycled cotton 20%Lining: Recycled polyester 65%, Recycled cotton 10%
Art. No.1024256004


In [17]:
text = str()

text = [text + line.text  for line  in elements]
text

['FitSlim fit',
 'CompositionShell: Cotton 99%, Spandex 1%Pocket lining: Polyester 65%, Cotton 35%',
 'More sustainable materialsShell: Recycled cotton 20%Lining: Recycled polyester 65%, Recycled cotton 10%',
 'Art. No.1024256004']

In [18]:
text_raw =' /'.join(text)
text_raw

'FitSlim fit /CompositionShell: Cotton 99%, Spandex 1%Pocket lining: Polyester 65%, Cotton 35% /More sustainable materialsShell: Recycled cotton 20%Lining: Recycled polyester 65%, Recycled cotton 10% /Art. No.1024256004'

In [19]:
# searching for words fit and composition in all text retrieved from products web page
for element in text:
    if 'fit' in element:
        fit = element
    if 'Composition' in element:
        composition = element
print(fit)
print(composition)

FitSlim fit
CompositionShell: Cotton 99%, Spandex 1%Pocket lining: Polyester 65%, Cotton 35%


### Price

In [46]:
driver.get("https://www2.hm.com/en_us/productpage.0979945001.html")

class_price = "ProductPrice-module--productItemPrice__2i2Hc"
element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )

# if element returns empty, try this other class
if element.text == '':
    class_price = "price.parbase"
    element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
    price = element.text
    
    if price == '':
        price = 'NA'
price

'$31.99$39.99'

### Joining Everything

In [47]:
df_comp = pd.DataFrame()

for idx, link in enumerate(df_prods['link']):
    # sku
    sku = link.split('.')[3]
    print('scraping page {}/{}: {}'.format( idx+1, len(df_prods), link))
    
    # load web page
    driver.get(link)
    
    # get price
    # try this class (for no promo days)
    class_price = "ProductPrice-module--productItemPrice__2i2Hc"
    element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )

    # if element returns empty, try this other class
    if element.text == '':
        class_price = "price.parbase"
        element = WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, class_price) ) )
        price = element.text
        
        if price == '':
            price = 'NA'
    
    # get product description   
    class_desc = "ProductDescription-module--descriptionText__1zy9P"      
    # test if description exists
    try: 
        content = WebDriverWait(driver, 10).until(EC.presence_of_element_located( (By.CLASS_NAME, class_desc) ))
        desc = content.text
    except:
        desc = 'NA'
    
    # get text
    class_text = 'ProductAttributesList-module--descriptionListItem__3vUL2'
    contents = WebDriverWait(driver, timeout=10).until( EC.presence_of_all_elements_located( (By.CLASS_NAME, class_text) ) )
    
    # concatenate all lines of text
    text = str()
    # list with all text
    text = [text + line.text  for line  in contents]

    # separate fit and composition from text
    # if fit or composition is not informed they'll return NA
    fit = 'NA'
    composition = 'NA'
    for element in text:
        if 'fit' in element:
            fit = element
        if 'Composition' in element:
            composition = element    
    
    # saving raw text
    text_raw =' /'.join(text)
    
    # saving results
    df_aux = pd.DataFrame( {'sku' : sku, 'price' : price, 'fit' : fit, 'composition' : composition, 'description' : desc ,'text' : text_raw,}, index = [0] )
    df_comp = pd.concat( [df_comp, df_aux], axis = 0 )     

df_comp.reset_index(inplace = True, drop = True)

scraping page 1/171: https://www2.hm.com/en_us/productpage.0427159001.html
scraping page 2/171: https://www2.hm.com/en_us/productpage.0427159002.html
scraping page 3/171: https://www2.hm.com/en_us/productpage.0427159003.html
scraping page 4/171: https://www2.hm.com/en_us/productpage.0427159004.html
scraping page 5/171: https://www2.hm.com/en_us/productpage.0427159005.html
scraping page 6/171: https://www2.hm.com/en_us/productpage.0427159006.html
scraping page 7/171: https://www2.hm.com/en_us/productpage.0427159007.html
scraping page 8/171: https://www2.hm.com/en_us/productpage.0427159008.html
scraping page 9/171: https://www2.hm.com/en_us/productpage.0427159010.html
scraping page 10/171: https://www2.hm.com/en_us/productpage.0427159011.html
scraping page 11/171: https://www2.hm.com/en_us/productpage.0427159017.html
scraping page 12/171: https://www2.hm.com/en_us/productpage.0427159022.html
scraping page 13/171: https://www2.hm.com/en_us/productpage.0427159023.html
scraping page 14/171:

In [48]:
df_comp.head()

,sku,price,fit,composition,description,text
0,0427159001,$31.99$39.99,FitSkinny fit,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste..."
1,0427159002,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste..."
2,0427159003,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste..."
3,0427159004,$31.99$39.99,FitSkinny fit,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ..."
4,0427159005,$31.99$39.99,FitSkinny fit,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste..."


# Data Parsing

### Composition

In [49]:
df_comp.head()

,sku,price,fit,composition,description,text
0,0427159001,$31.99$39.99,FitSkinny fit,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste..."
1,0427159002,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste..."
2,0427159003,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste..."
3,0427159004,$31.99$39.99,FitSkinny fit,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ..."
4,0427159005,$31.99$39.99,FitSkinny fit,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste..."


In [54]:
df_comp_aux = df_comp.copy()

comps = []
linings = []

for idx, text in enumerate(df_comp_aux['composition']):
    # case 1 pocket lining present
    if 'Pocket' in text:
        # regex = '(Shell: .*?=Pocket|Cotton.*(?=Pocket))'
        regex = 'Cotton.*(?=Pocket)'
        try:
            comp = re.findall( regex, text)[0]
        except:
            comp = 'NA'
    # case 2 pocket lining not present
    else:
        regex = '(Cotton.*(?=Lining)|Cotton.*(?=lining)|Cotton.*%)'
        try:
            comp = re.findall( regex, text)[0]
        except:
            comp = 'NA'
        # print(df_comp_aux.loc[idx, 'sku'] + '|' + text +' | ' + comp)
    
    # geting pocket composition:
    regex = '(?<=lining: ).*'
    try:
        lining = re.findall(regex, text)[0]
    except:
        lining = 'Not Informed'
    linings.append(lining)
    
    comps.append(comp)
df_comp_aux['comp'] = comps
df_comp_aux['lining'] = linings

In [55]:
df_comp_aux.head()

,sku,price,fit,composition,description,text,comp,lining
0,0427159001,$31.99$39.99,FitSkinny fit,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...","Cotton 91%, Polyester 7%, Spandex 2%",Not Informed
1,0427159002,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed
2,0427159003,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed
3,0427159004,$31.99$39.99,FitSkinny fit,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...","Cotton 99%, Spandex 1%",Not Informed
4,0427159005,$31.99$39.99,FitSkinny fit,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...","Cotton 72%, Polyester 20%, Modal 7%, Spandex 1%",Not Informed


In [64]:
df_comp_split = pd.DataFrame()

for composition in df_comp_aux['comp']:
    comp_list = composition.split(' ') 

    # creating a df of compositions
    df_aux = composition_to_df(comp_list)

    df_comp_split = pd.concat( [df_comp_split, df_aux], axis = 0 )

df_comp_split.reset_index(inplace = True, drop = True)

In [65]:

df_comp_aux = pd.concat( [df_comp_aux, df_comp_split], axis = 1 )
df_comp_aux.head()


,sku,price,fit,composition,description,text,comp,lining,Cotton,Polyester,Spandex,Modal,Elastomultiester
0,0427159001,$31.99$39.99,FitSkinny fit,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...","Cotton 91%, Polyester 7%, Spandex 2%",Not Informed,91,7,2,NaN,NaN
1,0427159002,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed,93,6,1,NaN,NaN
2,0427159003,$31.99$39.99,FitSkinny fit,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed,93,6,1,NaN,NaN
3,0427159004,$31.99$39.99,FitSkinny fit,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...","Cotton 99%, Spandex 1%",Not Informed,99,NaN,1,NaN,NaN
4,0427159005,$31.99$39.99,FitSkinny fit,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...","Cotton 72%, Polyester 20%, Modal 7%, Spandex 1%",Not Informed,72,20,1,7,NaN


In [67]:
df_comp_aux.tail()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Polyester,Spandex,Modal,Elastomultiester
166,1027852007,$23.99$29.99,FitRelaxed fit,CompositionShell: Cotton 100%Pocket lining: Co...,"Joggers in thick cotton denim. Regular waist, ...","SizeThe model is 185cm/6'1"" and wears a size M...",Cotton 100%,Cotton 100%,100,NaN,NaN,NaN,NaN
167,1048642001,$31.99$39.99,FitRegular fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%","Polyester 80%, Cotton 20%",99,NaN,1,NaN,NaN
168,1048642002,$31.99$39.99,FitRegular fit,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%",Cotton 100%,99,NaN,1,NaN,NaN
169,1049466001,$31.99$39.99,FitLoose fit,CompositionCotton 100%,"Jeans in thick cotton denim. Regular waist, zi...",FitLoose fit /CompositionCotton 100% /More sus...,Cotton 100%,Not Informed,100,NaN,NaN,NaN,NaN
170,1063426001,$39.99$49.99,NA,CompositionCotton 100%,Edition by is an uncompromising collection of ...,"SizeThe model is 188cm/6'2"" and wears a size 3...",Cotton 100%,Not Informed,100,NaN,NaN,NaN,NaN


### Fit

In [68]:
# positive lookbehind + words I'm searching + positive lookahead
regex = "((?<=Fit).*(?= fit)|NA)"

df_comp_aux['fit'] = df_comp_aux['fit'].apply(lambda x: re.findall(regex, x)[0] )
df_comp_aux.head()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Polyester,Spandex,Modal,Elastomultiester
0,0427159001,$31.99$39.99,Skinny,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...","Cotton 91%, Polyester 7%, Spandex 2%",Not Informed,91,7,2,NaN,NaN
1,0427159002,$31.99$39.99,Skinny,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed,93,6,1,NaN,NaN
2,0427159003,$31.99$39.99,Skinny,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed,93,6,1,NaN,NaN
3,0427159004,$31.99$39.99,Skinny,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...","Cotton 99%, Spandex 1%",Not Informed,99,NaN,1,NaN,NaN
4,0427159005,$31.99$39.99,Skinny,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...","Cotton 72%, Polyester 20%, Modal 7%, Spandex 1%",Not Informed,72,20,1,7,NaN


### Price

In [74]:
regex = "\$\d{2}.\d{2}\$\d{2}.\d{2}"
price = '$22.22$11.11'
price = '$31.99$39.99'

bool(re.match(regex, price ))

True

In [93]:
# df_comp_aux = df_comp_aux.copy()

# if there are 2 prices then there is a discount/promo
# regex = "\$\d{2}.\d{2}\$\d{2}.\d{2}"
regex = "\$\d+\.\d+\$\d+.\d+"
df_comp_aux['isPromo'] = df_comp_aux['price'].apply(lambda x: 1 if bool(re.match(regex, x)) else 0)

# first price
regex = "^\$\d+\.\d+"
df_comp_aux['firstPrice'] = df_comp_aux['price'].apply( lambda x: re.findall(regex, x)[0] )

# second price
regex = "\$\d+\.\d+$"
df_comp_aux['secondPrice'] = df_comp_aux['price'].apply( lambda x: re.findall(regex, x)[0] )

# removing
df_comp_aux['firstPrice'] = df_comp_aux['firstPrice'].apply(lambda x: x.strip('$')).astype(float)
df_comp_aux['secondPrice'] = df_comp_aux['secondPrice'].apply(lambda x: x.strip('$')).astype(float)

# 
df_comp_aux['finalPrice'] = df_comp_aux.apply( lambda x: x['firstPrice'] if x['firstPrice'] <= x['secondPrice'] else x['secondPrice'], axis =1 )
df_comp_aux['originalPrice'] = df_comp_aux.apply( lambda x: x['secondPrice'] if x['secondPrice'] >= x['firstPrice'] else x['firstPrice'], axis =1 )

df_comp_aux.drop(['firstPrice', 'secondPrice'], axis = 1, inplace = True)

In [95]:
# df_comp_aux[df_comp_aux.firstPrice == df_comp_aux.secondPrice]
df_comp_aux.head()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Polyester,Spandex,Modal,Elastomultiester,isPromo,originalPrice,finalPrice
0,0427159001,$31.99$39.99,Skinny,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...","Cotton 91%, Polyester 7%, Spandex 2%",Not Informed,91,7,2,NaN,NaN,1,39.99,31.99
1,0427159002,$31.99$39.99,Skinny,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed,93,6,1,NaN,NaN,1,39.99,31.99
2,0427159003,$31.99$39.99,Skinny,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...","Cotton 93%, Polyester 6%, Spandex 1%",Not Informed,93,6,1,NaN,NaN,1,39.99,31.99
3,0427159004,$31.99$39.99,Skinny,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...","Cotton 99%, Spandex 1%",Not Informed,99,NaN,1,NaN,NaN,1,39.99,31.99
4,0427159005,$31.99$39.99,Skinny,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...","Cotton 72%, Polyester 20%, Modal 7%, Spandex 1%",Not Informed,72,20,1,7,NaN,1,39.99,31.99


In [96]:
df_comp_aux[df_comp_aux['isPromo'] == True].tail()

,sku,price,fit,composition,description,text,comp,lining,Cotton,Polyester,Spandex,Modal,Elastomultiester,isPromo,originalPrice,finalPrice
166,1027852007,$23.99$29.99,Relaxed,CompositionShell: Cotton 100%Pocket lining: Co...,"Joggers in thick cotton denim. Regular waist, ...","SizeThe model is 185cm/6'1"" and wears a size M...",Cotton 100%,Cotton 100%,100,NaN,NaN,NaN,NaN,1,29.99,23.99
167,1048642001,$31.99$39.99,Regular,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%","Polyester 80%, Cotton 20%",99,NaN,1,NaN,NaN,1,39.99,31.99
168,1048642002,$31.99$39.99,Regular,"CompositionShell: Cotton 99%, Spandex 1%Pocket...",5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...","Cotton 99%, Spandex 1%",Cotton 100%,99,NaN,1,NaN,NaN,1,39.99,31.99
169,1049466001,$31.99$39.99,Loose,CompositionCotton 100%,"Jeans in thick cotton denim. Regular waist, zi...",FitLoose fit /CompositionCotton 100% /More sus...,Cotton 100%,Not Informed,100,NaN,NaN,NaN,NaN,1,39.99,31.99
170,1063426001,$39.99$49.99,NA,CompositionCotton 100%,Edition by is an uncompromising collection of ...,"SizeThe model is 188cm/6'2"" and wears a size 3...",Cotton 100%,Not Informed,100,NaN,NaN,NaN,NaN,1,49.99,39.99


### Headline

In [97]:
# removing whitespace characteres

df_prods['headline'] = df_prods['headline'].apply(lambda x: x.strip('\n\t ')) 
df_prods.headline.value_counts()

Slim Jeans                        33
Skinny Jeans                      26
Regular Jeans                     23
Trashed Skinny Jeans              15
Relaxed Jeans                     14
Slim Tapered Jeans                14
Hybrid Regular Tapered Joggers     7
Regular Tapered Crop Jeans         6
Skinny Cropped Jeans               5
Freefit® Slim Jeans                5
Relaxed Tapered Pull-on Jeans      4
Loose Jeans                        4
Hybrid Regular Denim Joggers       3
Regular Denim Joggers              3
Relaxed Pull-on Jeans              3
Relaxed Denim Joggers              2
Regular Bootcut Jeans              2
Loose Carpenter Jeans              1
Cotton Denim Jeans                 1
Name: headline, dtype: int64

In [98]:
df_final = pd.concat( [df_prods, df_comp_aux.drop('sku', axis = 1)], axis =1 )

# adding date time
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_final['date'] = now
df_final.head()

,sku,product_id,color_id,color,headline,link,price,fit,composition,description,...,lining,Cotton,Polyester,Spandex,Modal,Elastomultiester,isPromo,originalPrice,finalPrice,date
0,0427159001,0427159,001,Black denim,Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...,$31.99$39.99,Skinny,"CompositionCotton 91%, Polyester 7%, Spandex 2%",5-pocket jeans in washed stretch denim. Heavil...,...,Not Informed,91,7,2,NaN,NaN,1,39.99,31.99,2022-04-16 19:13:21
1,0427159002,0427159,002,Blue washed out,Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...,$31.99$39.99,Skinny,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,...,Not Informed,93,6,1,NaN,NaN,1,39.99,31.99,2022-04-16 19:13:21
2,0427159003,0427159,003,Denim blue,Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...,$31.99$39.99,Skinny,"CompositionCotton 93%, Polyester 6%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,...,Not Informed,93,6,1,NaN,NaN,1,39.99,31.99,2022-04-16 19:13:21
3,0427159004,0427159,004,Light denim blue,Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...,$31.99$39.99,Skinny,"CompositionCotton 99%, Spandex 1%",5-pocket jeans in washed stretch denim. Heavil...,...,Not Informed,99,NaN,1,NaN,NaN,1,39.99,31.99,2022-04-16 19:13:21
4,0427159005,0427159,005,Dark denim blue,Trashed Skinny Jeans,https://www2.hm.com/en_us/productpage.04271590...,$31.99$39.99,Skinny,"CompositionCotton 72%, Polyester 20%, Modal 7%...",5-pocket jeans in washed stretch denim. Heavil...,...,Not Informed,72,20,1,7,NaN,1,39.99,31.99,2022-04-16 19:13:21


In [99]:
df_final.columns

Index(['sku', 'product_id', 'color_id', 'color', 'headline', 'link', 'price',
       'fit', 'composition', 'description', 'text', 'comp', 'lining', 'Cotton',
       'Polyester', 'Spandex', 'Modal', 'Elastomultiester', 'isPromo',
       'originalPrice', 'finalPrice', 'date'],
      dtype='object')

In [100]:
#sku, product_id, color_id, color, fit, price, headline, 'cotton', 'polyester', 'elastane', 'elasterell_p', 'spandex' 'modal', 'viscose', pocket_lining, text
selected_cols = ['sku', 'product_id', 'color_id', 'color', 'fit', 'finalPrice', 'originalPrice', "headline", 'Cotton', 'Polyester', 'Spandex', 'Modal', 'Elastomultiester', 'isPromo', 'description', 'text', 'link', 'date'] 
df_final = df_final[selected_cols]

rename_cols = ['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price', 'original_price', "headline", 'cotton', 'polyester', 'spandex', 'modal', 'elastomultiester', 'is_promo', 'description', 'text', 'link', 'date'] 

final_cols = dict(zip(selected_cols, rename_cols))

df_final.rename(columns = final_cols, inplace = True )
df_final.head()



,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,spandex,modal,elastomultiester,is_promo,description,text,link,date
0,0427159001,0427159,001,Black denim,Skinny,31.99,39.99,Trashed Skinny Jeans,91,7,2,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
1,0427159002,0427159,002,Blue washed out,Skinny,31.99,39.99,Trashed Skinny Jeans,93,6,1,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
2,0427159003,0427159,003,Denim blue,Skinny,31.99,39.99,Trashed Skinny Jeans,93,6,1,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
3,0427159004,0427159,004,Light denim blue,Skinny,31.99,39.99,Trashed Skinny Jeans,99,NaN,1,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
4,0427159005,0427159,005,Dark denim blue,Skinny,31.99,39.99,Trashed Skinny Jeans,72,20,1,7,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21


# Data Saving

## Inserting data into local SQLite DB

In [103]:
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# df_comp.to_csv(path_or_buf='./backups/df_comp-{}.csv'.format(now), )

# saving df as a local backup
df_final.to_csv('../backups/df_backup-{}.csv'.format(now), index = False)

print('saved: df_backup-{}.csv'.format(now))

saved: df_backup-2022-04-16 20:55:06.csv


## Loading data from previously scraped data

In [104]:
# specifying dtypes
dtypes = { 'Art. No.': 'str', 'style_id' : 'str', 'color_id' : 'str' }

df_old = pd.read_csv('../previous_data.csv', dtype = dtypes)
df_old.head()

,Art. No.,style_id,color_id,color,Fit,Price,Composition,texts,Cotton,Polyester,Elastane,Elasterell-P,Modal,Viscose,link,date,day_number,date.1,date_rank
0,0690449001,0690449,001,Light denim blue/trashed,Skinny fit,16.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,1,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,11,2021-09-11 17:19:36,1
1,0690449002,0690449,002,Denim blue,Skinny fit,14.99,"Cotton 98%, Elastane 2%","Cotton 98%, Elastane 2%",98,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,11,2021-09-11 17:19:36,1
2,0690449006,0690449,006,Black/washed,Skinny fit,7.99,"Lining: Cotton 100% Cotton 98%, Elastane 2%","Cotton 100% Cotton 98%, Elastane 2%",100,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,11,2021-09-11 17:19:36,1
3,0690449007,0690449,007,Light denim blue,Skinny fit,14.99,"Cotton 98%, Elastane 2% Lining: Cotton 100%","Cotton 98%, Elastane 2% Lining",98,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,11,2021-09-11 17:19:36,1
4,0690449009,0690449,009,Black washed out,Skinny fit,19.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,1,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,11,2021-09-11 17:19:36,1


In [105]:
df_old.dtypes

Art. No.         object
style_id         object
color_id         object
color            object
Fit              object
Price           float64
Composition      object
texts            object
Cotton            int64
Polyester         int64
Elastane          int64
Elasterell-P      int64
Modal             int64
Viscose           int64
link             object
date             object
day_number        int64
date.1           object
date_rank         int64
dtype: object

In [106]:
drop_cols = ["day_number", "date.1", "date_rank"]

df_old.drop(columns = drop_cols, inplace = True)
df_old.head()

,Art. No.,style_id,color_id,color,Fit,Price,Composition,texts,Cotton,Polyester,Elastane,Elasterell-P,Modal,Viscose,link,date
0,0690449001,0690449,001,Light denim blue/trashed,Skinny fit,16.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,1,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
1,0690449002,0690449,002,Denim blue,Skinny fit,14.99,"Cotton 98%, Elastane 2%","Cotton 98%, Elastane 2%",98,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
2,0690449006,0690449,006,Black/washed,Skinny fit,7.99,"Lining: Cotton 100% Cotton 98%, Elastane 2%","Cotton 100% Cotton 98%, Elastane 2%",100,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
3,0690449007,0690449,007,Light denim blue,Skinny fit,14.99,"Cotton 98%, Elastane 2% Lining: Cotton 100%","Cotton 98%, Elastane 2% Lining",98,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
4,0690449009,0690449,009,Black washed out,Skinny fit,19.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,1,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36


In [107]:
df_old['color'].value_counts()

Denim blue                  1073
Light denim blue             928
Black                        495
Dark denim blue              443
Dark gray                    294
Dark blue                    251
White                        193
Pale denim blue              119
Gray                         117
Black/washed out             112
Black/No fade black          112
Light gray                   112
Light blue                   112
Midnight blue                 83
Blue                          67
Light denim blue/trashed      58
Black denim                   56
Black/trashed                 56
Black washed out              56
Black/washed                  56
Graphite gray                 55
Denim gray                    32
Cream                         28
Dark blue denim               28
Black washed-out              28
Gray denim                    28
Denim blue washed out         28
Black/bleached                28
Light denim gray/Trashed      28
Dark denim blue/trashed       28
Dark blue/

### Data Cleaning

In [110]:
# for comp in df_old['Composition']:
#     print(comp)

In [108]:
# saving linings
linings = []

regex = "(lining.*(?= Shell)|lining.* Cotton \d{1,}%.*(?=Cotton)|lining.*%|Lining.*Cotton \d{1,}%.*(?=Cotton))"

# searching linings according to regex
for string in df_old['Composition']:
    try:
        lining = re.findall(regex, string)[0]
        lining.strip('lining: Lining')
    except:
        lining = 'Not Informed'

    # gathering all linings for each product
    linings.append(lining)

In [109]:
df_old['lining'] = linings
df_old.head()

,Art. No.,style_id,color_id,color,Fit,Price,Composition,texts,Cotton,Polyester,Elastane,Elasterell-P,Modal,Viscose,link,date,lining
0,0690449001,0690449,001,Light denim blue/trashed,Skinny fit,16.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,1,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed
1,0690449002,0690449,002,Denim blue,Skinny fit,14.99,"Cotton 98%, Elastane 2%","Cotton 98%, Elastane 2%",98,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed
2,0690449006,0690449,006,Black/washed,Skinny fit,7.99,"Lining: Cotton 100% Cotton 98%, Elastane 2%","Cotton 100% Cotton 98%, Elastane 2%",100,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Lining: Cotton 100%
3,0690449007,0690449,007,Light denim blue,Skinny fit,14.99,"Cotton 98%, Elastane 2% Lining: Cotton 100%","Cotton 98%, Elastane 2% Lining",98,0,2,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed
4,0690449009,0690449,009,Black washed out,Skinny fit,19.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,1,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed


In [110]:
# cleaning fit column
df_old['Fit'] = df_old['Fit'].apply(lambda x: x.strip(' fit'))

In [111]:
# renaming columns

df_old.columns

new_cols = {'Art. No.':'sku' ,
 'style_id':'product_id' ,
 'color_id': 'color_id', 
 'color': 'color', 
 'Fit': 'fit', 
 'Price':'final_price' ,
'Composition': 'Composition', 
'texts':'text' ,
'Cotton': 'cotton', 
'Polyester':'polyester' ,
'Elastane':'elastane' ,
'Elasterell-P': 'elasterell_p', 
'Modal': 'modal', 
'Viscose': 'viscose', 
'link': 'link', 
'date': 'date'}

# add headling

In [112]:
# adding missing columns and renaming
df_old['headline'] = df_old['color']
df_old['original_price'] = df_old['Price']
df_old['is_promo'] = 0
df_old['description'] = 'Not Informed'


df_old.rename(columns = new_cols, inplace = True)

In [113]:
df_old.columns

Index(['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price',
       'Composition', 'text', 'cotton', 'polyester', 'elastane',
       'elasterell_p', 'modal', 'viscose', 'link', 'date', 'lining',
       'headline', 'original_price', 'is_promo', 'description'],
      dtype='object')

In [114]:
df_old.head()

,sku,product_id,color_id,color,fit,final_price,Composition,text,cotton,polyester,...,elasterell_p,modal,viscose,link,date,lining,headline,original_price,is_promo,description
0,0690449001,0690449,001,Light denim blue/trashed,Skinny,16.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,...,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed,Light denim blue/trashed,16.99,0,Not Informed
1,0690449002,0690449,002,Denim blue,Skinny,14.99,"Cotton 98%, Elastane 2%","Cotton 98%, Elastane 2%",98,0,...,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed,Denim blue,14.99,0,Not Informed
2,0690449006,0690449,006,Black/washed,Skinny,7.99,"Lining: Cotton 100% Cotton 98%, Elastane 2%","Cotton 100% Cotton 98%, Elastane 2%",100,0,...,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Lining: Cotton 100%,Black/washed,7.99,0,Not Informed
3,0690449007,0690449,007,Light denim blue,Skinny,14.99,"Cotton 98%, Elastane 2% Lining: Cotton 100%","Cotton 98%, Elastane 2% Lining",98,0,...,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed,Light denim blue,14.99,0,Not Informed
4,0690449009,0690449,009,Black washed out,Skinny,19.99,"Cotton 99%, Elastane 1%","Cotton 99%, Elastane 1%",99,0,...,0,0,0,https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36,Not Informed,Black washed out,19.99,0,Not Informed


In [115]:
selected_cols = ['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price',
                 'original_price', 'headline', 'cotton', 'polyester', 'elastane',
                 'elasterell_p', 'viscose', 'modal','is_promo', 'description', 
                 'text', 'link', 'date']

# 'elastane','elasterell_p', 'viscose'

df_old = df_old[selected_cols].copy()
df_old.head()

,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,elastane,elasterell_p,viscose,modal,is_promo,description,text,link,date
0,0690449001,0690449,001,Light denim blue/trashed,Skinny,16.99,16.99,Light denim blue/trashed,99,0,1,0,0,0,0,Not Informed,"Cotton 99%, Elastane 1%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
1,0690449002,0690449,002,Denim blue,Skinny,14.99,14.99,Denim blue,98,0,2,0,0,0,0,Not Informed,"Cotton 98%, Elastane 2%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
2,0690449006,0690449,006,Black/washed,Skinny,7.99,7.99,Black/washed,100,0,2,0,0,0,0,Not Informed,"Cotton 100% Cotton 98%, Elastane 2%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
3,0690449007,0690449,007,Light denim blue,Skinny,14.99,14.99,Light denim blue,98,0,2,0,0,0,0,Not Informed,"Cotton 98%, Elastane 2% Lining",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
4,0690449009,0690449,009,Black washed out,Skinny,19.99,19.99,Black washed out,99,0,1,0,0,0,0,Not Informed,"Cotton 99%, Elastane 1%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36


In [116]:
df_old.head()
df_final.head()

,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,spandex,modal,elastomultiester,is_promo,description,text,link,date
0,0427159001,0427159,001,Black denim,Skinny,31.99,39.99,Trashed Skinny Jeans,91,7,2,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
1,0427159002,0427159,002,Blue washed out,Skinny,31.99,39.99,Trashed Skinny Jeans,93,6,1,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
2,0427159003,0427159,003,Denim blue,Skinny,31.99,39.99,Trashed Skinny Jeans,93,6,1,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
3,0427159004,0427159,004,Light denim blue,Skinny,31.99,39.99,Trashed Skinny Jeans,99,NaN,1,NaN,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21
4,0427159005,0427159,005,Dark denim blue,Skinny,31.99,39.99,Trashed Skinny Jeans,72,20,1,7,NaN,1,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-16 19:13:21


In [118]:
# defining today scraped data as :
df_final_16_04 = df_final.copy()

df_final_16_04.isna().sum()

sku                   0
product_id            0
color_id              0
color                 0
fit                   0
final_price           0
original_price        0
headline              0
cotton                0
polyester           150
spandex              39
modal               170
elastomultiester    166
is_promo              0
description           0
text                  0
link                  0
date                  0
dtype: int64

In [120]:
# read all as string
dtypes = { 'sku' : 'str', 'product_id' : 'str', 'color_id' : 'str'}
df_final_13_04 = pd.read_csv('../backups/df_backup-2022-04-13 21:51:21.csv', dtype = dtypes)
df_final_13_04.rename(columns = {'original_rice' : 'original_price'}, inplace = True)
df_final_13_04.head()

,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,spandex,modal,elastomultiester,is_promo,description,text,link,date
0,0427159001,0427159,001,Black denim,Skinny,39.99,39.99,Trashed Skinny Jeans,91,7.0,2,NaN,NaN,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
1,0427159002,0427159,002,Blue washed out,Skinny,39.99,39.99,Trashed Skinny Jeans,93,6.0,1,NaN,NaN,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
2,0427159003,0427159,003,Denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,93,6.0,1,NaN,NaN,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
3,0427159004,0427159,004,Light denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,99,NaN,1,NaN,NaN,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
4,0427159005,0427159,005,Dark denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,72,20.0,1,7.0,NaN,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54


In [123]:
df_final_13_04['spandex'].unique()


array(['2', '1', nan, '2%Lining'], dtype=object)

In [124]:
spandex = df_final_13_04['spandex']
values = []
for element in spandex:
    if element == "2%Lining":
        values.append(2)
    else:
        values.append(element)

df_final_13_04['spandex'] = values
df_final_13_04['spandex'] = df_final_13_04['spandex'].astype(float)
df_final_13_04['spandex'] = df_final_13_04['spandex'].fillna(0)
df_final_13_04['spandex'] = df_final_13_04['spandex'].astype(int)


In [126]:
df_final_13_04.isna().sum()
df_final_16_04.isna().sum()


sku                   0
product_id            0
color_id              0
color                 0
fit                   0
final_price           0
original_price        0
headline              0
cotton                0
polyester           150
spandex              39
modal               170
elastomultiester    166
is_promo              0
description           0
text                  0
link                  0
date                  0
dtype: int64

### Merging all data and inserting it to MySQL on AWS

In [27]:
# # creating sqlalchemy engine for connection
# path = 'sqlite:///' + path_to_db
# engine = create_engine(path, echo=False)

# # creating a Session class
# Session = sessionmaker(bind=engine)

# # creating a session
# session = Session()
# query = """
# DROP TABLE IF EXISTS hm_showroom
# """

# try:
#     engine.execute(query)
#     session.commit()
# except:
#     session.rollback()

In [127]:
df_old.isna().sum()
df_old.shape

(5390, 19)

In [143]:
# filling NAs
df_final_13_04.head()
df_final_13_04.isna().sum()
df_final_13_04[df_final_13_04['fit'].isna()]

# # filling NAs
df_final_13_04.loc[df_final_13_04['fit'].isna()] = 'Not Informed'
df_final_13_04.fillna(value = 0, inplace = True)
df_final_13_04.isna().sum()


sku                 0
product_id          0
color_id            0
color               0
fit                 0
final_price         0
original_price      0
headline            0
cotton              0
polyester           0
spandex             0
modal               0
elastomultiester    0
is_promo            0
description         0
text                0
link                0
date                0
dtype: int64

In [144]:
df_final_13_04.head()

,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,spandex,modal,elastomultiester,is_promo,description,text,link,date
0,0427159001,0427159,001,Black denim,Skinny,39.99,39.99,Trashed Skinny Jeans,91,7.0,2,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 91%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
1,0427159002,0427159,002,Blue washed out,Skinny,39.99,39.99,Trashed Skinny Jeans,93,6.0,1,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
2,0427159003,0427159,003,Denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,93,6.0,1,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 93%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
3,0427159004,0427159,004,Light denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,99,0,1,0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 99%, Spandex ...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54
4,0427159005,0427159,005,Dark denim blue,Skinny,39.99,39.99,Trashed Skinny Jeans,72,20.0,1,7.0,0,0,5-pocket jeans in washed stretch denim. Heavil...,"FitSkinny fit /CompositionCotton 72%, Polyeste...",https://www2.hm.com/en_us/productpage.04271590...,2022-04-13 21:49:54


In [145]:
df_final_16_04.isna().sum()
df_final_16_04[df_final_16_04['fit'].isna()]

# # filling NAs
df_final_16_04.loc[df_final_16_04['fit'].isna()] = 'Not Informed'
df_final_16_04.fillna(value = 0, inplace = True)
df_final_16_04.isna().sum()

sku                 0
product_id          0
color_id            0
color               0
fit                 0
final_price         0
original_price      0
headline            0
cotton              0
polyester           0
spandex             0
modal               0
elastomultiester    0
is_promo            0
description         0
text                0
link                0
date                0
dtype: int64

In [146]:
df_old.columns

Index(['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price',
       'original_price', 'headline', 'cotton', 'polyester', 'elastane',
       'elasterell_p', 'viscose', 'modal', 'is_promo', 'description', 'text',
       'link', 'date'],
      dtype='object')

In [147]:
df_final.columns

Index(['sku', 'product_id', 'color_id', 'color', 'fit', 'final_price',
       'original_price', 'headline', 'cotton', 'polyester', 'spandex', 'modal',
       'elastomultiester', 'is_promo', 'description', 'text', 'link', 'date'],
      dtype='object')

In [148]:
df_merged = pd.concat([df_old, df_final_13_04, df_final_16_04], axis = 0)
df_merged.head()

# changing columns order:
cols = ['sku','product_id','color_id','color','fit','final_price','original_price','headline','cotton','polyester', 'elastane', 'spandex','elastomultiester', 'elasterell_p','viscose','modal','is_promo','description','text','link','date']
df_merged = df_merged[cols].copy()

df_merged.isna().sum()

sku                    0
product_id             0
color_id               0
color                  0
fit                    0
final_price            0
original_price         0
headline               0
cotton                 0
polyester              0
elastane             348
spandex             5390
elastomultiester    5390
elasterell_p         348
viscose              348
modal                  0
is_promo               0
description            0
text                   0
link                   0
date                   0
dtype: int64

In [149]:
# filling NAs
df_merged.fillna(value = 0, inplace = True)

df_merged.isna().sum()

sku                 0
product_id          0
color_id            0
color               0
fit                 0
final_price         0
original_price      0
headline            0
cotton              0
polyester           0
elastane            0
spandex             0
elastomultiester    0
elasterell_p        0
viscose             0
modal               0
is_promo            0
description         0
text                0
link                0
date                0
dtype: int64

In [150]:
df_merged.head()


,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,...,spandex,elastomultiester,elasterell_p,viscose,modal,is_promo,description,text,link,date
0,0690449001,0690449,001,Light denim blue/trashed,Skinny,16.99,16.99,Light denim blue/trashed,99,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 99%, Elastane 1%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
1,0690449002,0690449,002,Denim blue,Skinny,14.99,14.99,Denim blue,98,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 98%, Elastane 2%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
2,0690449006,0690449,006,Black/washed,Skinny,7.99,7.99,Black/washed,100,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 100% Cotton 98%, Elastane 2%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
3,0690449007,0690449,007,Light denim blue,Skinny,14.99,14.99,Light denim blue,98,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 98%, Elastane 2% Lining",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36
4,0690449009,0690449,009,Black washed out,Skinny,19.99,19.99,Black washed out,99,0,...,0,0,0.0,0.0,0,0,Not Informed,"Cotton 99%, Elastane 1%",https://www2.hm.com/en_us/productpage.06904490...,2021-09-11 17:19:36


In [151]:
df_merged.shape

(5738, 21)

In [152]:
df_merged.spandex.unique()


array([0, 2, 1, 'Not Informed', '2', '1'], dtype=object)

In [154]:
# removing bug
df_merged.spandex.unique()
df_merged[df_merged.spandex == 'Not Informed']
df_merged = df_merged[ df_merged['spandex'] != 'Not Informed'].copy()


In [155]:
# converting dtypes
df_merged['final_price'] = df_merged['final_price'].astype(float)
df_merged['original_price'] = df_merged['original_price'].astype(float)
df_merged['cotton'] = df_merged['cotton'].astype(int)
df_merged['polyester'] = df_merged['polyester'].astype(int)
df_merged['elastane'] = df_merged['elastane'].astype(int)
df_merged['spandex'] = df_merged['spandex'].astype(int)
df_merged['elastomultiester'] = df_merged['elastomultiester'].astype(int)
df_merged['elasterell_p'] = df_merged['elasterell_p'].astype(int)
df_merged['viscose'] = df_merged['viscose'].astype(int)
df_merged['modal'] = df_merged['modal'].astype(int)
df_merged['is_promo'] = df_merged['is_promo'].astype(int)
df_merged['date'] = pd.to_datetime(df_merged['date'], errors = 'coerce')

In [167]:
# checking if there are no bugs or errors
df_merged.cotton.unique()
df_merged['polyester'].unique()
df_merged['elastane'].unique()
df_merged['spandex' ].unique()
df_merged['elastomultiester' ].unique()
df_merged['elasterell_p' ].unique()
df_merged['viscose' ].unique()
df_merged['modal'].unique()
df_merged['is_promo'].unique()

array([0, 1])

In [168]:
df_merged.dtypes

sku                         object
product_id                  object
color_id                    object
color                       object
fit                         object
final_price                float64
original_price             float64
headline                    object
cotton                       int64
polyester                    int64
elastane                     int64
spandex                      int64
elastomultiester             int64
elasterell_p                 int64
viscose                      int64
modal                        int64
is_promo                     int64
description                 object
text                        object
link                        object
date                datetime64[ns]
dtype: object

In [176]:
df_merged[df_merged['original_price'] < df_merged['final_price'] ].shape

(0, 21)

In [177]:
df_merged.tail()

,sku,product_id,color_id,color,fit,final_price,original_price,headline,cotton,polyester,...,spandex,elastomultiester,elasterell_p,viscose,modal,is_promo,description,text,link,date
166,1027852007,1027852,007,Denim blue,Relaxed,23.99,29.99,Relaxed Denim Joggers,100,0,...,0,0,0,0,0,1,"Joggers in thick cotton denim. Regular waist, ...","SizeThe model is 185cm/6'1"" and wears a size M...",https://www2.hm.com/en_us/productpage.10278520...,2022-04-16 19:13:21
167,1048642001,1048642,001,Denim blue,Regular,31.99,39.99,Regular Bootcut Jeans,99,0,...,1,0,0,0,0,1,5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...",https://www2.hm.com/en_us/productpage.10486420...,2022-04-16 19:13:21
168,1048642002,1048642,002,Denim black,Regular,31.99,39.99,Regular Bootcut Jeans,99,0,...,1,0,0,0,0,1,5-pocket jeans in stretch cotton denim with a ...,"FitRegular fit /CompositionShell: Cotton 99%, ...",https://www2.hm.com/en_us/productpage.10486420...,2022-04-16 19:13:21
169,1049466001,1049466,001,Denim blue,Loose,31.99,39.99,Loose Carpenter Jeans,100,0,...,0,0,0,0,0,1,"Jeans in thick cotton denim. Regular waist, zi...",FitLoose fit /CompositionCotton 100% /More sus...,https://www2.hm.com/en_us/productpage.10494660...,2022-04-16 19:13:21
170,1063426001,1063426,001,Light indigo blue,NA,39.99,49.99,Cotton Denim Jeans,100,0,...,0,0,0,0,0,1,Edition by is an uncompromising collection of ...,"SizeThe model is 188cm/6'2"" and wears a size 3...",https://www2.hm.com/en_us/productpage.10634260...,2022-04-16 19:13:21


In [181]:
# reading credentials

secrets_json = open('../secrets/secrets.json')
secrets = json.load(secrets_json)

dialect =   secrets["dialect"]
driver =    secrets["driver"]
host =      secrets["host"]
username =  secrets["username"]
password =  secrets["password"]
port =      secrets["port"]
database =  secrets["database"]

url = "{}+{}://{}:{}@{}:{}/{}".format(dialect, driver, username, password, host, port, database)
# engine = create_engine(url = url, echo = True) #, pool_pre_ping = True

In [182]:
# Inserting processed data into MySQL DB

# creating sqlalchemy engine for connection
engine = create_engine(url, echo=True)

# creating a Session class
Session = sessionmaker(bind=engine)

# creating a session
session = Session()

In [186]:
# testing case a new column is added

try:
    # adding data
    df_merged.to_sql('hm_showroom', con = engine, if_exists='append', index = False)

    # committing changes
    session.commit()
except:
    try:
        # in case scraped data returns with a new column, it will be added to a new table
        table_name = "hm_showroom_backup-{}".format(datetime.now().strftime("%Y-%m-%d"))
        df_merged.to_sql( table_name, con = engine, if_exists='append', index = False)

        session.commit()
    except:  
        # if even this fails, undo everything      
        session.rollback()

finally:
    session.close()

# USE THIS SCRIPT TO PREVENT FAILS

2022-04-16 21:21:59,065 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-04-16 21:21:59,066 INFO sqlalchemy.engine.Engine [cached since 278.7s ago] {'table_schema': 'humberto_personal_projects', 'table_name': 'hm_showroom'}
2022-04-16 21:21:59,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-16 21:21:59,119 INFO sqlalchemy.engine.Engine 
CREATE TABLE hm_showroom (
	sku TEXT, 
	product_id TEXT, 
	color_id TEXT, 
	color TEXT, 
	fit TEXT, 
	final_price FLOAT(53), 
	original_price FLOAT(53), 
	headline TEXT, 
	cotton BIGINT, 
	polyester BIGINT, 
	elastane BIGINT, 
	spandex BIGINT, 
	elastomultiester BIGINT, 
	elasterell_p BIGINT, 
	viscose BIGINT, 
	modal BIGINT, 
	is_promo BIGINT, 
	description TEXT, 
	text TEXT, 
	link TEXT, 
	date DATETIME
)


2022-04-16 21:21:59,122 INFO sqlalchemy.engine.Engine [no key 0.00325s] {}
2022-04-16 21:21:59,168 INFO sqlalchemy.engine.Engine COMMI

## TODO

- clean script
- check datatypes and use datetime for date column
- automate data insertion
- create db on AWS DONE
- create EC2 to run script
- create Lambda with cronjob to run script